## Running Qiskit Aqua algorithms using measurement error mitigation

### Introduction
To execute quantum circuits as part of aqua algorithms, we define a QuantumInstance which holds the specification for the respective backend. If we want to use measurement error mitigation for the results of the quantum circuits runs, we can put a measurement error fitting instance into the QuantumInstance. Thus, the backend defined in the QuantumInstance may be viewed as a measurement error mitigated backend.

This notebook illustrates how to run an aqua algorithm on the example of searching for the ground state of a Hamiltonian with VQE.

First, we need to import the libraries that provide backends as well as the classes that are needed to run the algorithm.

In [1]:
# Import qiskit functions and libraries
from qiskit import Aer, IBMQ
from qiskit.circuit.library import TwoLocal
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.operators import X, Y, Z, I, CX, T, H, S, PrimitiveOp
from qiskit.providers.aer import noise

# Import error mitigation functions
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter

Then, we initialize the instances that are needed to run the algorithm.

In [2]:
# Initialize Hamiltonian
h_op =  (-1.0523732 * I^I) + \
                (0.39793742 * I^Z) + \
                (-0.3979374 * Z^I) + \
                (-0.0112801 * Z^Z) + \
                (0.18093119 * X^X)
# Initialize trial state
var_form = TwoLocal(h_op.num_qubits, ['ry', 'rz'], 'cz', reps=3, entanglement='full')
# Initialize optimizer
optimizer = SPSA(max_trials=350)
# Initialize algorithm to find the ground state
algo = VQE(h_op, var_form, optimizer)

Here, we define our backend which for reasons of illustration - is given by an Aer simulated noise model.
The use of a real quantum backend provided IBMQ is given in the commented part of the code.

In [4]:
# Generate a noise model
noise_model = noise.NoiseModel()
for qi in range(h_op.num_qubits):
    read_err = noise.errors.readout_error.ReadoutError([[0.9, 0.1],[0.25,0.75]])
    noise_model.add_readout_error(read_err, [qi])    
# Initialize the backend configuration using measurement error mitigation with a QuantumInstance 
qi_noise_model_qasm = QuantumInstance(backend=Aer.get_backend('qasm_simulator'), noise_model=noise_model, shots=8000,
                                      measurement_error_mitigation_cls=CompleteMeasFitter,
                                      measurement_error_mitigation_shots=8000)
# Intialize your TOKEN and provider with 
# provider = IBMQ.get_provider(...)
# qi_noise_model_ibmq = QuantumInstance(backend=provider = provider.get_backend(backend_name)), shots=8000,
#                                       measurement_error_mitigation_cls=CompleteMeasFitter, measurement_error_mitigation_shots=8000)

Finally, we can run the algorithm with a measurement error mitigated, shot-based simulated backend that uses a noise model.

In [5]:
# Run the algorithm
result = algo.run(qi_noise_model_qasm)
print(result)

{'optimal_parameters': {Parameter(θ[0]): -2.967486134198413, Parameter(θ[10]): -0.19061803266013988, Parameter(θ[11]): 0.7459430131412589, Parameter(θ[12]): 5.435352187816858, Parameter(θ[13]): -1.3927124973544567, Parameter(θ[14]): -1.2324538224586679, Parameter(θ[15]): 1.3105694915283719, Parameter(θ[1]): 5.109506548516348, Parameter(θ[2]): 4.517334392435216, Parameter(θ[3]): -3.27797107685917, Parameter(θ[4]): 2.084102108557699, Parameter(θ[5]): 0.21391707391242598, Parameter(θ[6]): -2.7014199772262866, Parameter(θ[7]): -0.6321797796657677, Parameter(θ[8]): 2.709123060013344, Parameter(θ[9]): -7.2118967795711155}, 'optimal_point': array([-2.96748613, -0.19061803,  0.74594301,  5.43535219, -1.3927125 ,
       -1.23245382,  1.31056949,  5.10950655,  4.51733439, -3.27797108,
        2.08410211,  0.21391707, -2.70141998, -0.63217978,  2.70912306,
       -7.21189678]), 'optimal_value': -1.8549568901838998, 'optimizer_time': 72.27436900138855, 'eigenvalue': (-1.8549568901838998+0j), 'eige